In [ ]:
import json
import pandas as pd

In [15]:
# Ouverture et conversion des fichiers concernés en objets python
df1 = pd.read_csv("distinct_mouvement.csv") 
df2 = pd.read_csv("Artists.csv") 
with open("art_subject_data.json", "r") as f:
    data = json.load(f)

# Pour des DataFrames df1 et df2, création de sets de paires clés-valeurs pour associer chaque terme à son identifiant Wikidata, et pouvoir y accéder facilement plus tard
movement_dict = {row["mouvementLabel"].lower(): row["WikiId"] for _, row in df1.dropna(subset=["WikiId"]).iterrows()}
artist_dict = {row["DisplayName"].lower(): row["Wiki QID"] for _, row in df2.dropna(subset=["Wiki QID"]).iterrows() if " " in row["DisplayName"]}


# Initialisation d'une liste vide utile pour y ajouter les entrées préparées et enrichies
filtered_data = [] 


# Parcours du JSON d'OpenLibrary 
for item in data:
    subjects = item.get("subject", []) 
    if not isinstance(subjects, list): 
        continue

    title = item.get("title", "").lower()

    # Recherche de matchs 
    subject_matches = [movement_dict[subject.lower()] for subject in subjects if subject.lower() in movement_dict] # Entre les mouvements artistiques Wikidata et les Subjects d'OpenLibrary
    artist_matches = [artist_dict[artist] for artist in artist_dict if artist in title] # Entre les noms d'artistes du dataset du MoMA et les titres des ouvrages d'OpenLibrary

    # Création de nouveaux dictionnaires enrichis des identifiants wikidata correspondants s'ils existent
    new_item = {
        "key": item.get("key"),
        "title": item.get("title"),
        "authors": item.get("authors"),
        "first_publish_year": item.get("first_publish_year"),
        "wikidataSubjectID": subject_matches, 
        "wikidataArtistID": artist_matches  
    }

    filtered_data.append(new_item)


# Dump de la liste enrichie dans un fichier JSON et vérifications
with open("enriched_openlibrary.json", "w", encoding="utf-8") as f:
    json.dump(filtered_data, f, indent=4, ensure_ascii=False)

if len(filtered_data) == len(data):
    print(f"✅ JSON d'origine et modifié ont le même nombre d'entrées ({len(filtered_data)})")
else:
    print(f"⚠ Erreur : Nombre d'entrées différent.\nOriginal : {len(data)}\nModifié : {len(filtered_data)}")


✅ JSON d'origine et modifié ont le même nombre d'entrées (116761)
